In [3]:
using HDF5, EzXML, Glob, Dates, Plots, Optim, LineSearches, LsqFit

datapath = "/Users/jjc/Documents/SSC/CSF Data/selected_recordings"
path = pwd();
savepath = "/Users/jjc/CSF/"
files = glob("*.hdf5", datapath);

# for filename in files

# filename =  datapath*"/INF_20200909122333_INF2.hdf5"
# filename = datapath*"/INF_20190708150215_INF2.hdf5"
# filename = datapath*"/INF_20191010113919_INF2.hdf5"
filename = datapath*"/INF_20190620142837_INF2.hdf5"


fid = h5open(filename, "r")

# Get attributes outside of datasets/groups
recording_end_time = read_attribute(fid, "dataEndTime")
recording_start_time = read_attribute(fid, "dataStartTime")
dur = read_attribute(fid, "duration")
dur = split(dur[1], " ")
recording_duration = parse(Int64, dur[1])

# Begin handling data in XML string - some files do not have infusion test output
xml_obj = fid["aux/ICM+/icmtests"]
xml_data = read(xml_obj)
xml_string = String(xml_data[1])

data = parsexml(xml_string)
icm_data = root(data)
vars = elements(icm_data)

SingleAnalysis = firstelement(vars[2])
SingleAnalysis = elements(SingleAnalysis)
Variables = elements(SingleAnalysis[1])
Selections = elements(SingleAnalysis[2])
Parameters = elements(SingleAnalysis[3])
Results = elements(SingleAnalysis[4])

inf_rate = parse(Float64, Parameters[1]["Value"])

function parseXML(Parameters, Selections)
    
    selections = Dict{String, String}
    results = Dict{String, Float64}
    
    for i in 1:length(Selections)
        sel = elements(Selections[i])
        sel_name = Selections[i]["Name"]
        sel_st = sel[1]["StartTime"]
        sel_en = sel[1]["EndTime"]  

        sel_st = DateTime(sel_st, dateformat"dd/mm/yyyy HH:MM:SS")
        sel_en = DateTime(sel_en, dateformat"dd/mm/yyyy HH:MM:SS")

        selections =  merge!(selections, Dict(sel_name => [sel_st, sel_en]))
    end

    for i in 1:length(Results)
        res = elements(Results[i]);
        res_name = Results[i]["Name"]
        name_surrogate = split(res_name, " ");

        # Some parameters name have units e.g. [mmHg] but Dict field naming does not support strings with non-alphanumeric chars
        if isletter(name_surrogate[2][1])
            res_name = name_surrogate[1]*"_"*name_surrogate[2];
        else
            res_name = name_surrogate[1];
        end

        res_val = Results[i]["Value"]
        results =  merge!(results, Dict(res_name => parse(Float64, res_val)))
    end
    return results, selections
end

results, selections = parseXML(Parameters, Selections);

# This dereferencing is pain, there has to be a more elegant solution
t_series_ds = fid["summaries/minutes/"]
t_series = t_series_ds[:]

numsamples = length(t_series)

# Pre-allocate
ICP = zeros(numsamples)
AMP = zeros(numsamples)
timestamp = zeros(numsamples)
P0 = zeros(numsamples)
AMP_P = zeros(numsamples)

# Dereferencing named tuple...
for i in 1:numsamples
    ICP[i] = t_series[i].ICP
    AMP[i] = t_series[i].AMP
    timestamp[i] = t_series[i].datetime
    P0[i] = t_series[i].P0
    AMP_P[i] = t_series[i].AMP_P
end

# Model equation
function mfun(X, t)
    I_b = X[1]
    E = X[2]
    P_0 = X[3]
    It = I_b + I_inf
    ΔP = P_b - P_0
    y = It * ΔP / (I_b + (I_inf * exp(-E*It*t))) + P_0;
    return y
end

# Sum squared error function
function sqerror(X)
    δ = zeros(6) 
    err = 0.0
    for i = 1:length(Pm)
        tᵢ = (i-1)/6 # or -2/-3?
        predᵢ = mfun(X,tᵢ)
        err += (Pm[i] - predᵢ)^2
    end
    δ[1] = maximum([0.0, lower[1]-X[1]])
    δ[2] = maximum([0.0, X[1]-upper[1]])
    δ[3] = maximum([0.0, lower[2]-X[2]])
    δ[4] = maximum([0.0, X[2]-upper[2]])
    δ[5] = maximum([0.0, lower[3]-X[3]])
    δ[6] = maximum([0.0, X[3]-upper[3]])

    dist = vcat(lower.-X, X.-upper) # Distances between the constraint and the current value
    h = dist.^κ

    penalty = C*(sum(δ.^2)+sum(h))
    # penalty = sum(r.*exp.(δ)./sum(exp.(δ))) # Softmax
    return err + penalty
end

# Plotting
function plot_model(I_b, E, P_0, Pss, Rcsf)

    println("Estimated parameters:\nIₐ = $I_b [mL/min]\n"*"E = $E [mmHg/mL]\n"*"P₀ = $P_0 [mmHg]\n")

    P_m = ones(numsamples)
    P_m *= P_b
    P_model = zeros(numsamples)
    ICPm = zeros(infusion_end_frame-infusion_start_frame)
    Ib = (P_b - Pss)/Rcsf
    It = I_b + I_inf
    ΔP = P_b - P_0

    for i = infusion_start_frame:infusion_end_frame
        tᵢ = (i-infusion_start_frame)/6
        y = It * ΔP / (I_b + (I_inf * exp(-E*It*tᵢ))) + P_0;
        P_model[i] = y
    end

    ICPm = P_model[infusion_start_frame:infusion_end_frame]
    P_m[infusion_start_frame:infusion_end_frame] = ICPm
    P_m[infusion_end_frame+1:end] .= ICPm[end]
    # P_m[infusion_end_frame+1:end] .= P_p

    plateau_end=numsamples
    vline([infusion_start_frame], background=:transparent, legend=:outertopright, linestyle=:dash, linecolor=:white, linewidth=1, label="Start of infusion")
    plot!(ICP[1:plateau_end], linecolor=:cadetblue, linewidth=2, label="Measured")
    h = plot!(P_m[1:plateau_end], linecolor=:orange, linewidth=2, linestyle=:dash, xlims=[1, plateau_end], ylims=[0,40], 
    xlabel="Time [min]", ylabel="ICP [mmHg]",xticks = ([0:30:plateau_end;], [0:30:plateau_end;]./10),
    label = "Model")
end;

In [2]:
start_time = DateTime(recording_start_time[1], dateformat"yyyy/mm/dd HH:MM:SS")
end_time = DateTime(recording_end_time[1], dateformat"yyyy/mm/dd HH:MM:SS")
infusion_start_frame = round(Int, (selections["Infusion"][1] - start_time).value / 10000) # 10,000 because /1,000 from ms -> s, and sampling every 10s (0.1Hz) 
infusion_end_frame = round(Int, (selections["Infusion"][2] - start_time).value / 10000)
plateau_start = round(Int, (selections["Plateau"][1] - start_time).value / 10000)
plateau_end= round(Int, (selections["Plateau"][2] - start_time).value / 10000)

P_p  = results["ICP_plateau"]

rec_dur = end_time - start_time # in milliseconds
rec_dur_s = rec_dur.value / 1000

# Get parameters in legible forms - not neccessary, done for clarity

# Constants:
global I_inf = inf_rate;
global P_b = results["ICP_baseline"]
T = [0:numsamples-1...]
global T *= 1 / 6

Pm = ICP[infusion_start_frame:infusion_end_frame]# Data truncated only to the period of infusion

# Estimates
I_b = results["CSF_production"]
global E = 0.0
try
    E = results["Elasticity"]
catch
    global E = 0.11
end
P_0 = results["Pss"]
println("Iₐ = $I_b [mL/min]\n" * "E = $E [mL/mmHg]\n" * "P₀ = $P_0 [mmHg]\n")

# Optimization setup
# x[1] = I_b
# x[2] = E
# x[3] = P_0
# x0 = [I_b, E, P_0]
X = I_b, E, P_0

x0 = [0.3, 0.1, 5] # Starting point
lower = [0.1, 0.01, 0.0]
upper = [1.0, 1.0, 50.0]

C = 10^1 #Cᵢ
κ = 2

res = optimize(sqerror, x0, LBFGS())
I_b, E, P_0 = Optim.minimizer(res)
R = Dict("LBFGS" => Optim.minimum(res))
plot_model(I_b, E, P_0)

Iₐ = 0.91 [mL/min]
E = 0.11 [mL/mmHg]
P₀ = -5.0 [mmHg]



LoadError: MethodError: no method matching plot_model(::Float64, ::Float64, ::Float64)
[0mClosest candidates are:
[0m  plot_model(::Any, ::Any, ::Any, [91m::Any[39m, [91m::Any[39m) at In[1]:138

In [4]:
using Interact
## Interact.WebIO.install_jupyter_nbextension() # might be helpful if you see `WebIO` warnings in Jupyter
moving_average(vs,n) = [sum(@view vs[i:(i+n-1)])/n for i in 1:(length(vs)-(n-1))]
dsampf = 40
gg = moving_average(ICP, dsampf)
g0 = zeros(length(ICP))
g0 .+= P_b
g0[Int(dsampf/2):Int(dsampf/2)+length(gg)-1] = gg
# Pm = g0
P_0 = results["Pss"]

@manipulate throttle=.05 for E=0:.01:1.0, Pss=-5.0:.1:P_b, Rcsf=0:1:50

    P_m = ones(numsamples)
    P_m *= P_b
    P_model = zeros(numsamples)
    ICPm = zeros(infusion_end_frame-infusion_start_frame)
    Ib = (P_b - Pss)/Rcsf
    It = I_b + I_inf
    ΔP = P_b - P_0
    
    for i = infusion_start_frame:infusion_end_frame
        tᵢ = (i-infusion_start_frame-1)/6
        y = It * ΔP / (I_b + (I_inf * exp(-E*It*tᵢ))) + P_0;
        P_model[i] = y
    end
    
    ICPm = P_model[infusion_start_frame:infusion_end_frame]
    P_m[infusion_start_frame:infusion_end_frame] = ICPm
    P_m[infusion_end_frame+1:end] .= ICPm[end]
    
    Plots.plot([ICP[1:plateau_end], P_m[1:plateau_end], g0], background=:transparent,linecolor=[:cadetblue :orange :green], linewidth=2, linestyle=[:solid :dash], xlims=[1, plateau_end], ylims=[0,40], 
    xlabel="Time [min]", ylabel="ICP [mmHg]", xticks = ([0:30:plateau_end;], [0:30:plateau_end;]./10),legend=:outertopright)
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["E"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 101, :min => 1, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{Observables.AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing), "index" => (Observable{Any} with 2 listeners. Value:
51, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/Users/jjc/.julia/packages/Knockout/3gjS1/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/jjc/.julia/packages/Knockout/3gjS1/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/jjc/.julia/packages/InteractBase/h8Fwe/src/../assets/all.js"), Asset("css", nothing, "/Users/jjc/.julia/packages/InteractBase/h8Fwe/src/../assets/style.css"), Asset("css", nothing, "/Users/jjc/.julia/packages/Interact/pVyN2/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x0000000290c3b820, Task (runnable) @0x0000000290c3b820), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"0.0\",\"0.01\",\"0.02\",\"0.03\",\"0.04\",\"0.05\",\"0.06\",\"0.07\",\"0.08\",\"0.09\",\"0.1\",\"0.11\",\"0.12\",\"0.13\",\"0.14\",\"0.15\",\"0.16\",\"0.17\",\"0.18\",\"0.19\",\"0.2\",\"0.21\",\"0.22\",\"0.23\",\"0.24\",\"0.25\",\"0.26\",\"0.27\",\"0.28\",\"0.29\",\"0.3\",\"0.31\",\"